In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.optimize as opt
import scipy.stats as stats
import statsmodels.api as sm

rf_rate = 0.046

In [2]:
# data: 
xls_dict  = pd.read_excel('data/trading-game-data-08112023.xlsx', sheet_name=None)
#data = pd.read_excel('trading-game-data-20102023.xlsx', sheet_name='price')
info_df = xls_dict['info'][['Ticker', 'RBICS Economy']]
index_price_df = xls_dict['index-price']
price_df = xls_dict['price']
size_df = xls_dict['size']
price_to_book_df = xls_dict['price-to-book']
turnover_df = xls_dict['turnover']

info_df = info_df.set_index('Ticker')

## ESG ratings
We will be using the esg ratings from https://www.gigasheet.com/sample-data/sp-500-esg-risk-ratings. Some companies are present in the dataframe therefore we will be using the sector median (and merged based on the info tab in the Trading game datasheet).

In [4]:
esgratings_df = pd.read_csv('data/esg data.csv')
esgratings_df = esgratings_df[['Symbol', 'Sector', 'Total ESG Risk score']]
sector_medians = esgratings_df.groupby(esgratings_df['Sector']).median()

esgratings_df.set_index('Symbol', inplace=True)
esgratings_df = esgratings_df.T

full_esg_df = pd.DataFrame(index=info_df.index)
full_esg_df['ESG Score'] = np.nan  # Initialize all scores as NaN

# Iterate over all companies in the S&P 500
for company in full_esg_df.index:
    if company in esgratings_df.index:
        # Use the actual ESG score if available
        full_esg_df.loc[company, 'ESG Score'] = esgratings_df.loc[company, 'Total ESG Risk score']
    else:
        # Retrieve the sector from info_df
        sector = info_df.loc[company, 'RBICS Economy']
        # Use the sector median if the sector is available
        if sector in sector_medians.index:
            full_esg_df.loc[company, 'ESG Score'] = sector_medians.loc[sector, 'Total ESG Risk score']

# Handling cases where the sector is unknown or no median is available
full_esg_df['ESG Score'].fillna(full_esg_df['ESG Score'].median(), inplace=True)
full_esg_df['Inverted_ESG'] = -full_esg_df['ESG Score']
full_esg_df

,ESG Score,Inverted_ESG
Ticker,,
A,22.0,-22.0
AAL,23.0,-23.0
AAPL,16.0,-16.0
ABBV,22.0,-22.0
ABNB,22.0,-22.0
...,...,...
YUM,22.0,-22.0
ZBH,22.0,-22.0
ZBRA,16.0,-16.0


In [16]:
# Log returns
price_df = xls_dict['price'].reset_index()
price_df = price_df.drop('index', axis = 1)
price_df['Date'] = pd.to_datetime(price_df['Date'])
price_df.set_index('Date', inplace=True)


# stock price infomration
log_price = np.log(price_df)
daily_returns = log_price.pct_change()
daily_returns = daily_returns.drop(daily_returns.index[0])
expected_returns = daily_returns.mean()
risk = daily_returns.std()
average_returns = daily_returns.mean() * len(daily_returns)


# Calculate SMB

In [17]:
#Calculating the factors per date, SMB
#SMB needs average returns of stocks
size_df = xls_dict['size'].reset_index() 
size_df = size_df.drop('index', axis = 1)
size_df['Date'] = pd.to_datetime(size_df['Date'])
size_df.set_index('Date', inplace=True)

average_returns_top =[]
average_returns_low = []

for i in range(len(size_df)-1):
    top_stocks = size_df.iloc[i].nlargest(int(len(size_df.iloc[i])*0.3)).index
    low_stocks = size_df.iloc[i].nsmallest(int(len(size_df.iloc[i])*0.3)).index
    average_returns_top.append(daily_returns.iloc[i][top_stocks].mean())
    average_returns_low.append(daily_returns.iloc[i][low_stocks].mean())

ave_return_df = pd.DataFrame({'top': average_returns_top, 'low': average_returns_low})
smb = ave_return_df['low'] - ave_return_df['top']

smb

0     -0.000100
1      0.003525
2     -0.000259
3      0.001052
4      0.001523
         ...   
209   -0.002807
210    0.002660
211    0.005202
212   -0.003049
213   -0.000140
Length: 214, dtype: float64

# HML

In [8]:
# Now HML
#calculate market to book. As price to book / price = 1/book value bepalen
#make new dataframe with price divided by price to book
price_to_book_df = xls_dict['price-to-book'].reset_index()
price_to_book_df = price_to_book_df.drop('index', axis = 1)
price_to_book_df['Date'] = pd.to_datetime(price_to_book_df['Date'])
price_to_book_df.set_index('Date', inplace=True)

bookvalue_df = price_df / price_to_book_df
btm_df = bookvalue_df / size_df
btm_df

average_returns_top_hml =[]
average_returns_low_hml = []

for i in range(len(size_df)-1):
    top_stocks = btm_df.iloc[i].nlargest(int(len(size_df.iloc[i])*0.3)).index
    low_stocks = btm_df.iloc[i].nsmallest(int(len(size_df.iloc[i])*0.3)).index
    average_returns_top_hml.append(daily_returns.iloc[i][top_stocks].mean())
    average_returns_low_hml.append(daily_returns.iloc[i][low_stocks].mean())

ave_return_hml_df = pd.DataFrame({'top': average_returns_top_hml, 'low': average_returns_low_hml})
hml = ave_return_df['top'] - ave_return_df['low']
hml

0     -0.000175
1     -0.012152
2     -0.000187
3     -0.000964
4     -0.006410
         ...   
209    0.011296
210   -0.006039
211   -0.018426
212    0.011327
213    0.000621
Length: 214, dtype: float64

# MKT_rf

In [18]:
#now Mkt-rf
#Calculate risk free rate

#Calculate market returns
index_price_df = xls_dict['index-price'].reset_index()
index_price_df = index_price_df.drop('index', axis = 1)
index_price_df['Date'] = pd.to_datetime(index_price_df['Date'])
index_price_df.set_index('Date', inplace=True)
daily_market_returns = index_price_df.pct_change()
mkt_rf = daily_market_returns - rf_rate
mkt_rf = mkt_rf.drop(mkt_rf.index[0])
mkt_rf

,S&P 500
Date,
2023-01-03,-0.050001
2023-01-04,-0.038461
2023-01-05,-0.057646
2023-01-06,-0.023159
2023-01-09,-0.046768
...,...
2023-11-01,-0.035494
2023-11-02,-0.027141
2023-11-03,-0.036606


In [ ]:
#make a multivariate regression using variables SMB, HML, Mkt-rf and ESG score

In [43]:
daily_returns
    
#get the lenght of the columns
n = len(daily_returns.columns)

df_params = pd.DataFrame(index = daily_returns.columns, columns = ['alpha', 'beta_smb', 'beta_hml', 'beta_mkt'])
df_signif = pd.DataFrame(index = daily_returns.columns, columns = ['alpha', 'beta_smb', 'beta_hml', 'beta_mkt'])

for i in range(0, len(daily_returns.columns)):
    #regress the daily returns of the first stock on the factors and ESG score
    y = daily_returns.iloc[:,i]
    x = np.column_stack((smb, hml, mkt_rf))
    x = sm.add_constant(x)
    model = sm.OLS(y,x)
    results = model.fit()

    for j in range(0, len(df_params.columns)):
        df_params.iloc[i, j] = results.params[j]
    #get the t-values
    for j in range(0, len(df_signif.columns)):
        df_signif.iloc[i, j] = results.tvalues[j]
# df_params

df_signif
# export the dataframes to excel
df_params.to_excel('params.xlsx')
df_signif.to_excel('signif.xlsx')


In [42]:
#Sort the stocks on beta_smb value
df_params = df_params.astype(float)

df_params_smb = df_params.sort_values(by = 'beta_smb', ascending = False)
df_params_smb
df_smb_top = df_params_smb.iloc[0:200]
df_smb_top

,alpha,beta_smb,beta_hml,beta_mkt
ZION,0.072725,1.742393,-1.742393,1.597695
CMA,0.071186,1.699586,-1.699586,1.574448
KEY,0.066668,1.674089,-1.674089,1.475793
PARA,0.061818,1.656781,-1.656781,1.352607
VFC,0.038022,1.557134,-1.557134,0.861158
...,...,...,...,...
BX,0.077450,0.335003,-0.335003,1.668212
L,0.029088,0.334375,-0.334375,0.623388
ALL,0.027161,0.332950,-0.332950,0.595502
IPG,0.037303,0.332350,-0.332350,0.828135
